## To make things simpler, the datasets will be renamed: <br>
alik05_forest-fire-dataset = dataset1 <br> <br>
elmadafri_the-wildfire-dataset = dataset2 <br> <br>
kutaykutlu_forest-fire = dataset3 <br> <br>
mohnishsaiprasad_forest-fire-images = dataset4 <br> <br>
phylake1337_fire-dataset = dataset5 <br> <br>

In [ ]:
import os
import numpy as np
from PIL import Image

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
#path_to_dataset_folder = "C:/Users/RoscaMitrut/Desktop/Forest-Fire-Detection/datasets"
path_to_dataset_folder = ""

In [ ]:
dataset1_path = path_to_dataset_folder + "datasets/alik05_forest-fire-dataset/Forest Fire Dataset/"
dataset2_path = path_to_dataset_folder + "datasets/elmadafri_the-wildfire-dataset/the_wildfire_dataset_2n_version/"
dataset3_path = path_to_dataset_folder + "datasets/kutaykutlu_forest-fire/"
dataset4_path = path_to_dataset_folder + "datasets/mohnishsaiprasad_forest-fire-images/Data/"
dataset5_path = path_to_dataset_folder + "datasets/phylake1337_fire-dataset/fire_dataset/"

In [ ]:
def load_data_paths(dataset_path,folders_labels):
    dataset_x = []
    dataset_y = []
    for folder,label in folders_labels:
        try:
            for f in os.scandir(dataset_path+folder):
                if f.is_file() and f.name.endswith((".jpg", ".png")):
                    dataset_x.append(dataset_path+folder+"/"+f.name)
                    dataset_y.append(label)
        except:
            break                
    return dataset_x,dataset_y

# Dataset1

In [ ]:
dataset1_X, dataset1_Y = load_data_paths(dataset1_path,[("Training/fire",1),("Training/nofire",0),("Testing/fire",1),("Testing/nofire",0)])

len(dataset1_X),len(dataset1_Y)

# Dataset2

In [ ]:
dataset2_X, dataset2_Y = load_data_paths(dataset2_path,[("test/fire",1),("test/nofire",0),("train/fire",1),("train/nofire",0),("val/fire",1),("val/nofire",0)])

len(dataset2_X),len(dataset2_Y)

# Dataset3

In [ ]:
dataset3_X, dataset3_Y = load_data_paths(dataset3_path,[("train_fire",1)])

len(dataset3_X),len(dataset3_Y)

# Dataset4

In [ ]:
dataset4_X, dataset4_Y = load_data_paths(dataset4_path,[("Test_Data/Fire",1),("Test_Data/Non_Fire",0),("Train_Data/Fire",1),("Train_Data/Non_Fire",0)])

len(dataset4_X),len(dataset4_Y)

# Dataset5

In [ ]:
dataset5_X, dataset5_Y = load_data_paths(dataset5_path,[("fire_images",1),("non_fire_images",0)])

len(dataset5_X),len(dataset5_Y)

# Splitting and merging datasets for training

In [ ]:
dataset_X = np.array(dataset1_X + dataset2_X + dataset3_X + dataset4_X + dataset5_X)
dataset_Y = np.array(dataset1_Y + dataset2_Y + dataset3_Y + dataset4_Y + dataset5_Y)

def shuffle_data(X, Y):
    indices = np.random.permutation(len(X))
    return X[indices], Y[indices]

dataset_X,dataset_Y = shuffle_data(dataset_X,dataset_Y)

X_train,X_test,Y_train,Y_test = train_test_split(dataset_X,dataset_Y,test_size=0.16)
X_train,X_val,Y_train,Y_val = train_test_split(X_train,Y_train,test_size=0.19)

In [ ]:
del dataset1_X,dataset1_Y,dataset2_X,dataset2_Y,dataset3_X,dataset3_Y,dataset4_X,dataset4_Y,dataset5_X,dataset5_Y,dataset_X,dataset_Y
del dataset1_path,dataset2_path,dataset3_path,dataset4_path,dataset5_path,path_to_dataset_folder

In [ ]:
def load_image(file_name, target_size = (256, 256)):
    raw = tf.io.read_file(file_name)
    tensor = tf.io.decode_image(raw, channels=3, expand_animations=False)
    tensor = tf.image.resize(tensor, target_size)
    tensor = tf.cast(tensor, tf.float32) / 255.0
    return tensor

def create_dataset(file_paths, labels, batch_size=32, target_size=(256, 256)):
    file_paths_tensor = tf.constant(file_paths)
    labels_tensor = tf.constant(labels, dtype=tf.float32)
    
    dataset = tf.data.Dataset.from_tensor_slices((file_paths_tensor, labels_tensor))
    
    def _load_and_preprocess(path, label):
        image = load_image(path, target_size)
        return image, label
    
    dataset = dataset.map(_load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset
    

In [ ]:
train_dataset = create_dataset(X_train, Y_train, batch_size=32, target_size=(256, 256))
test_dataset = create_dataset(X_test, Y_test, batch_size=32, target_size=(256, 256))
val_dataset = create_dataset(X_val, Y_val, batch_size=32, target_size=(256, 256))

In [ ]:
for data in train_dataset.take(1):
    print("Dataset element type:", type(data))
    print("Number of elements:", len(data))
    if len(data) == 2:
        images, labels = data
        print("Images shape:", images.shape)
        print("Labels shape:", labels.shape)
        print("Sample labels:", labels.numpy())
        
del data,images,labels


# Training

# MODEL 1

In [ ]:
model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

model.summary()

In [ ]:
history = model.fit(train_dataset, epochs=7, validation_data=val_dataset, verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['precision'], label = 'precision')
plt.plot(history.history['recall'], label = 'recall')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['val_precision'], label = 'val_precision')
plt.plot(history.history['val_recall'], label = 'val_recall')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

y_pred = model.predict(test_dataset).flatten()
y_pred = np.round(y_pred)
plot_confusion_matrix(Y_test, y_pred)

# MODEL 2

In [ ]:

model2 = models.Sequential([
    layers.Rescaling(1./255, input_shape=(256, 256, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.15),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    layers.Dropout(0.15),

    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.30),
    layers.Dense(1, activation='sigmoid')
])


model2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

model2.summary()


In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs_model'),
]

In [ ]:
history2 = model2.fit(train_dataset, epochs=50, validation_data=val_dataset, verbose=1, callbacks=callbacks)

In [ ]:
plt.plot(history2.history['loss'], label='loss')
plt.plot(history2.history['accuracy'], label='accuracy')
plt.plot(history2.history['precision'], label = 'precision')
plt.plot(history2.history['recall'], label = 'recall')
plt.plot(history2.history['val_loss'], label = 'val_loss')
plt.plot(history2.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history2.history['val_precision'], label = 'val_precision')
plt.plot(history2.history['val_recall'], label = 'val_recall')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.show()

y_pred = model2.predict(test_dataset).flatten()
y_pred = np.round(y_pred)
plot_confusion_matrix(Y_test, y_pred)

# Saving/Loading models

In [ ]:
model.save("fire_detection_model.h5")

In [ ]:
model2.save("fire_detection_model2.h5")

In [ ]:
loaded_model = models.load_model("fire_detection_model.h5")

In [ ]:
lloaded_model = models.load_model("fire_detection_model2.h5")